# Исследование надежности заемщиков

**Цель исследования**

Проверить, как влияет семейное положение и количество детей клиентов на факт погашения кредита в срок, а также подготовить данные для построения модели кредитного скоринга

**Ход исследования**  

Кредитный отдел банка предоставил для исследования датасет data.csv с описанием данных столбцов. О качестве данных ничего неизвестно. 

## Содержание

1. [Обзор данных](#start)  
2. [Предобработка данных](#preproccessing)   
    2.1 [Удаление дубликатов](#del_dub)  
    2.2 [Заполнение пропусков](#fill_miss)   
    2.3 [Проверка данных на аномалии и исправления](#anomaly_check)  
    2.4 [Выводы](#preproc_conclusion)  
3. [Исследование](#research)  
    3.1 [Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма](#add_dict)   
    3.2 [Категоризация данных](#categorization)
      * [Категоризация дохода](#income)  
      * [Категоризация целей кредита](#purpose)  
      * [Категоризация количества детей](#children)  
4. [Нахождение зависимостей](#relations)  
5. [Итоги исследования](#results)  
6. [Выводы и ответы на вопросы заказчика](#answers)  
7. [Общий вывод](#total)

## <a id="start"></a> 1. Обзор данных 

Получим общую информацию о данных

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv('data.csv')
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Из файла с описанием данных: 

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В датасете 12 столбцов, ошибок в типах данных не обнаружено.  
Обнаружено одинаковое количество пропущенных значений в столбце с общим трудовым стажем `days_employed` и столбце с доходом `total_income`.

## <a id="preproccessing"></a> 2. Предобработка данных

### 2.1 Удаление дубликатов <a id="del_dub"></a>

Проверим таблицу на предмет явных дубликатов

In [4]:
df.duplicated().sum()

54

В данных обнаружены следующие артефакты:

1. 54 явных строки-дубликата
2. Неявные дубликаты из-за разного регистра в столбце с образованием `education`
3. Пропущенные значения в столбцах `days_employed` и `total_income`
4. Лишнее неинформативное количество символов после запятой в столбце `total_income`
5. Отрицательные значения в столбце с трудовым стажем `days_employed`

Во-первых, избавимся от явных дубликатов

In [5]:
df = df.drop_duplicates().reset_index(drop=True) #удалим явные дубликаты с присвоением новых индексов
df.duplicated().sum()

0

Во-вторых, избавимся от неявных дубликатов в колонке `education`

In [6]:
df.education.sort_values().unique() #выведем уникальные значения столбца

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

Проблема только в регистре, поэтому достаточно просто привести все значения к нижнему регистру.

In [7]:
df.education = df.education.str.lower()
df.education.sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

### 2.2 Заполнение пропусков  <a id="fill_miss"></a>

Посчитаем количество пропущенных значений

In [8]:
df.isna().sum()

children               0
days_employed       2120
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2120
purpose                0
dtype: int64

Видим, что пропущенных значений одинковое количество в столбцах `days_employed ` и `total_income `

In [9]:
#Найдем долю пропущенных значений
print(f'Доля пропущенных значений: {df.total_income.isna().sum() / df.education.count():.0%}')

Доля пропущенных значений: 10%


10% - это много, мы не можем себе позволить просто удалить такое количество строк. Поэтому их нужно заполнить. Оптимальный вариант - заполнить средним по стоблцу, но использовать среднее арифметическое нежелательно, так как способ очень чувствителен к выбросам и может существенно снизить достоверность данных. В данном случае медиана - лучшее решение, которое не исказит общее представление, даже если среди значений встретятся выбросы. 

In [10]:
df.total_income.fillna(df.total_income.median(), inplace=True) #Заполним пропуски столбца "total_income" медианными значениями
df.total_income = df.total_income.astype('int') #также изменим тип данных столбца "total_income" на "int"

df.info() #проверим результаты

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21471 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21471 non-null  int64  
 3   education         21471 non-null  object 
 4   education_id      21471 non-null  int64  
 5   family_status     21471 non-null  object 
 6   family_status_id  21471 non-null  int64  
 7   gender            21471 non-null  object 
 8   income_type       21471 non-null  object 
 9   debt              21471 non-null  int64  
 10  total_income      21471 non-null  int32  
 11  purpose           21471 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


In [11]:
df.head(10) #проверим

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [12]:
df.duplicated().sum()

17

Интересное наблюдение, что после заполнения медианным значением столбца total_income в таблице снова появились 17 строк - явных дубликатов. Избавимся и от них.

In [13]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [14]:
df.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Осталось также заполнить медианой значения в столбце `days_employed`, однако есть проблема. В столбце есть очвидно некорректные значения (300 000 рабочих дней), а также отрицательные значения. Разберемся повнимательней.

### 2.3 Проверка данных на аномалии и исправления <a id="anomaly_check"></a>

In [15]:
#рабочих дней не может быть отрицательное число, поэтому проверим количество и долю некорректных значений
correct_count = df[df['days_employed'] >= 0]['days_employed'].count()
wrong_count = df[df['days_employed'] < 0]['days_employed'].count()
print(f'Положительных значений: {correct_count}')
print(f'Отрицательных значений: {wrong_count}')
print(f'Доля отрицательных значений: {(wrong_count / (correct_count + wrong_count)):.0%}')

Положительных значений: 3445
Отрицательных значений: 15906
Доля отрицательных значений: 82%


Положительных значений - всего пятая часть. Посмотрим поподробнее, что это за значения

In [16]:
positive_strings = df[df['days_employed'] >= 0]
positive_strings.head(15) #выведем 15 строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456,операции с коммерческой недвижимостью
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805,на проведение свадьбы
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342,автомобили
56,0,370145.087237,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141,образование
71,0,338113.529892,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929,автомобили
78,0,359722.945074,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127,сделка с автомобилем


Мы видим, что все значения больше 300 000 (это более 820 лет (причем только рабочих!😨). Уточнить причину и запросить корректные данные нет возможности, поэтому преположим, что во всех положительных значениях количество дней - некорректно.

Проверим эту теорию.  

За отсечку адекватного (максимально возможного) количества рабочих дней возьмем число 21900 (60 лет)

In [17]:
positive_series = positive_strings['days_employed']
if positive_series.loc[positive_series > 21900].count() == positive_series.count():
    print("Все положительные значения некорректны, теория верна")
else:
    print("Есть корректные значения, нужно больше анализа")

Все положительные значения некорректны, теория верна


Все положительные значения в столбце с количеством рабочих дней оказались некорректными. Логично предположить, что все отрицательные значения - корректные. Проверим!

In [18]:
negative_strings = df[df['days_employed'] < 0]
negative_strings.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья


In [19]:
negative_series = negative_strings['days_employed']
if negative_series.loc[negative_series <= 21900].count() == negative_series.count():
    print("Все отрицательные значения корректны, теория верна")
else:
    print("Есть некорректные значения, нужно больше анализа")

Все отрицательные значения корректны, теория верна


Итак, мы выяснили, что положительные значения некорректные, отрицательные же - напротив - все корректные, сделаем их положительными. 

Также предположим, что все положительные значения больше 300 000 - указаны в часах вместо дней. Переведем их в дни, а также заполним медианой все NaN'ы в этом столбце

In [20]:
df.loc[df['days_employed'] < 0, 'days_employed'] = abs(df.loc[df['days_employed'] < 0, 'days_employed'])
df.loc[df['days_employed'] > 21900, 'days_employed'] = (df.loc[df['days_employed'] > 21900, 'days_employed']) / 24
df.days_employed.fillna(df.days_employed.median(), inplace=True)
display(df.head(10))
display(df.isna().sum())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Отлично, пропущенных значений больше нет, а некорреткно большие значения дней переведены в корректные

### 2.4 Выводы  <a id="preproc_conclusion"></a>

Итак, на этапе предобработки мы обработали все ошибки и недочеты в данных, а именно:

* Удалили явные строки-дубликаты
* Убрали неявные дубликаты в столбце с образованием `education` (привели к одному регистру)
* Исправили пропущенные значения в столбце `total_income` с помощью медианы, а также привели к целочисленному формату
* Провели исследование отрицательных значений в столбце с трудовым стажем `days_employed`. Выяснили, что корректными являются отрицатаельные значения, некорректные также заполнили медианой, также привели значения к целочисленному формату


* Очевидно, что в случае с регистром - причина возникновения - исключительно человеческий фактор. Каждый указывал образование по-разному. 
* Длинные дробные части чисел `days_employed` и `total_income` - напротив - результат вычисления машин
* Пропуски в данных мы заполнили именно медианным значением по соответствующему столбцу, так как для чистоты выводов нам нужно именно серединное значение, не чувствительное к выбросам, чем грешит среднее арифметическое.
* Появление отрицательные значений в данных, которые, к тому же, согласно проведенному мини-исследованию, оказались корректными в отличие от положительных, скорее всего, также появились в следствие некорректной выгрузки, причиной которой могли стать и ошибки в вычислениях, и некорректный код, и просто человеческая ошибка

*теперь, после того, как данные очищены, можно переходить непосредственно  к исследованию*

## 3. Исследование <a id="research"></a>

### 3.1 Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма. <a id="add_dict"></a>

Нужно проверить, как влияют семейное положение и образование на факт возврата кредита.

Создадим два датафрейма-словаря, к которым можно будет обращаться по идентификатору. И уберем эти "текстовые" стоблцы из основного датафрейма

In [21]:
education_df = df[['education_id', 'education']] 
family_status_df = df[['family_status_id', 'family_status']]

In [22]:
df.drop(columns=['education', 'family_status'], axis=1, inplace=True)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### 3.2 Категоризация данных <a id="categorization"></a>

* **Категоризация дохода** <a id="income"></a>

Проведем ***категоризацию*** данных.  

Создадим новый столбец `total_income_category` со следующими категориями:

* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Для этого напишем функцию

In [23]:
def income_group(income):
    if income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    return 'A'

In [24]:
df['total_income_category'] = df.total_income.apply(income_group)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


* **Категоризация целей кредита** <a id="purpose"></a>

А также категоризируем цель получения кредита на: 

* операции с автомобилем,
* операции с недвижимостью,
* проведение свадьбы,
* получение образования.

In [25]:
def purpose_group(purpose):
    if "авто" in purpose or "машин" in purpose:
        return "операции с автомобилем"
    if "недвиж" in purpose or "жил" in purpose:
        return "операции с недвижимостью"
    if "свадьб" in purpose:
        return "проведение свадьбы"
    if "образован" in purpose:
        return "получение образования"
    return "иная цель"

In [26]:
df['purpose_category'] = df.purpose.apply(purpose_group)
df.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


* **Категоризация количества детей** <a id="children"></a>

Также категоризируем количество детей. Сначала посчитаем количество значений, чтобы понять как сгруппировать значения

In [27]:
df.groupby('children')['children'].count()

children
-1        47
 0     14091
 1      4808
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [28]:
def children_group(children):
    if children == 0 or children == -1:
        return 'нет детей'
    if children == 1:
        return '1 ребенок'
    if children == 2:
        return '2 ребенка'
    return 'многодетная семья'

In [29]:
df['children_group'] = df.children.apply(children_group)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_group
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,1 ребенок
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,1 ребенок
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,нет детей
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,многодетная семья
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,нет детей


## 4. Нахождение зависимостей <a id="relations"></a>

Теперь проверим, как влияют на факт возврата кредита отдельные признаки

Используем среднее арифметическое, чтобы получить среднее значение возврата кредита по каждой сформированной группе. 

Значение **1** в столбце `debt` значит, что у клиента была задолженность, Значение **0** - задолженностей не было, т.е. чем больше будет полученное значение, тем меньше в среднем данная группа возвращала кредит в срок.

Напишем функцию для проверки групп

In [30]:
def pivot_repayment(group: str):
    return df.pivot_table(index=group, values='debt', aggfunc='mean')

In [31]:
#список всех групп для проверки
groups = ['children_group', 'education_id', 'family_status_id', 'gender', 'income_type', 'total_income_category', 'purpose_category']

In [32]:
#Выведем все результаты
for group in groups:
    display(pivot_repayment(group))

,debt
children_group,
1 ребенок,0.092346
2 ребенка,0.094542
многодетная семья,0.085526
нет детей,0.075258


,debt
education_id,
0,0.052952
1,0.089902
2,0.091398
3,0.109929
4,0.000000


,debt
family_status_id,
0,0.075452
1,0.093471
2,0.065693
3,0.071130
4,0.097509


,debt
gender,
F,0.070128
M,0.102624
XNA,0.000000


,debt
income_type,
безработный,0.500000
в декрете,1.000000
госслужащий,0.059025
компаньон,0.074045
пенсионер,0.056412
предприниматель,0.000000
сотрудник,0.095724
студент,0.000000


,debt
total_income_category,
A,0.080000
B,0.070621
C,0.084915
D,0.060000
E,0.090909


,debt
purpose_category,
операции с автомобилем,0.093590
операции с недвижимостью,0.072334
получение образования,0.092200
проведение свадьбы,0.080034


## 5. Итоги исследования <a id="results"></a>

### Количество детей

Разница между группой с одним ребенком и двумя - несущественна, однако бездетные клиенты возвращают кредиты чаще. У многодетных семей показатель также лучше, но таких клиентов меньше. Поэтому с уверенностью можно заключить лишь то, что на возврат кредита в срок влияет не столько количество детей, сколько их наличие. 

### Уровень образования

Очевидно, что видна прямая зависимость вероятности возврата кредита от образования. Клиенты с высшим образованием возвращают кредиты намного лучше остальных групп. Далее - по нисходящей - чем ниже уровень образования, тем больше показателей задолженности по кредитам. А у 6 клиентов с ученой степенью вообще не было ни одной просрочки. 

### Семейное положение

Группы `женат/замужем`, `вдовец / вдова	` и `в разводе` имеют показатели по возвратам лучше, чем `неженатые/незамужние` или клиенты в `гражданском браке`. Возможно, причина в корреляции данных групп с показателем возраста. 

### Пол

Женщины возвращают кредиты лучше мужчин. Точка😑

### Уровень дохода

Корреляции показателя возврата кредитов с уровенм дохода не обнаружено. Группа с доходом 30-50т имеют показатель возврата выше, чем группа до 30т. Группа 50-200т возвращают примерно также как группа 1млн+. На основании имеющихся данных нельзя утверждать, что имеются зависимости данных показателей.

### Цель кредита

Охотнее всего возвращаются ипотеки, чуть ниже показатель у свадебных кредитов, и хуже всего с кредитной историей у студентов и автолюбителей.

## 6. Выводы и ответы на вопросы заказчика <a id="answers"></a>

Мы провели исследование и ответили на вопросы, поставленные заказчиком.

1. Есть ли зависимость между количеством детей и возвратом кредита в срок?  

Нельзя утверждать о зависимости между *количеством* детей и возвратом кредита, однако совершенно точно на вероятность возврата кредита влияет *наличие* детей как таковых.

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

Да, люди, состоящие (состоявшие) в официальном браке, возвращают кредиты чаще, чем люди "без штампа в паспорте". 

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Нет, такой зависимости не обнаружено.

4. Как разные цели кредита влияют на его возврат в срок?

* Чем выше образование, тем выше показатель возврата.  
* Женщины возвращают кредиты лучше мужчин
* Ипотеки и свадебные кредиты возвращаются лучше, чем кредиты на машину и обучение 

## 7. Общий вывод <a id="total"></a>

С небольшой долей допущения, можно заключить, что чем ответственней цель кредита, чем выше вероятность его возврата. Кредиты на жилье возвращаются чаще, чем кредиты на автомобили и свадьбы. Однако несмотря на это, кредиты на образование имеют не лучшие показатели возврата. Все-таки видимо студентам не хватает ответственности. 

☔☔☔